In [4]:
import pandas as pd
import numpy as np
from get_soup import *

import sys
sys.path.append("..")  # Adds higher directory to python modules path.
import util as util
import re


In [7]:
def get_table(soup, selector = '#mw-content-text > div > div.table-wide > div > table'):

    ''' 
    Extract the table from the html file
    parameters:
        soup: the soup object from the html file
        selector: the selector for the table in the html file
    returns:
        df_out: the dataframe of the table
    '''

    # Extract the table from the html file
    # Works for units and buildings

    # Find the table using the provided selector
    table = soup.select_one(selector)
    # print(table)

    # Data Storage
    data = []

    # Extract the header row
    header_row = table.thead.tr

    # Extract the header values
    header_values = [th.get_text(strip=True).lower() for th in header_row.find_all('th')]

    # change the name of some of the columns so it is easier to understand what they mean and to make the names standardized
    replacements = {
        'b':'building',
        'a': 'age',
        'at': 'attack',
        'infantry unit': 'unit',
        'archer unit': 'unit',
        'cavalry unit': 'unit',
        'ship': 'unit',
        'ra': 'range',
        'rof': 'rate of fire',
        'los': 'line of sight',
        'sp': 'speed',
        'tt': 'training time',
        'ci': 'civilization availability',
        'fd': 'frame delay',
        'ac': 'accuracy',
        'f':'food',
        'g':'gold',
        'ar': 'melee armor',
        'ma': 'melee armor',
        'pa': 'pierce armor',
        'ga': 'garrison capacity',
        'ct': 'construction time',
        'sz': 'size'
        }

    # replace the header values with the replacements 
    header_values = [replacements.get(x,x) for x in header_values]

    # Find all the rows in the table body
    rows = table.tbody.find_all('tr')

    # Extract the rows and columns from the table

    for row in rows:
        cols = row.find_all('td')
        cols_text = [col.get_text(strip=True) for col in cols]
        data.append(cols_text)

    # Combine the header and data
    df_out = pd.DataFrame(data, columns = header_values)

    # Make the first column the index. The index will be the name of the unit/building
    df_out.set_index(df_out.columns[0], inplace=True)
    # Make index lower case
    df_out.index = df_out.index.str.lower()
    
    return df_out


The next cell scrapes all the necessary data and saves each call in a dictionary so we don't need to scrape the data again every time.

In [4]:
# Infantry
url = 'https://ageofempires.fandom.com/wiki/Infantry_units_(Age_of_Empires_II)'
soup_infantry = get_data_from_wiki(url)

# Archers
url = 'https://ageofempires.fandom.com/wiki/Archer_units_(Age_of_Empires_II)'
soup_archer = get_data_from_wiki(url)

# Cavalry
url = 'https://ageofempires.fandom.com/wiki/Cavalry_units_(Age_of_Empires_II)'
soup_cavalry = get_data_from_wiki(url)

# Navy
url = 'https://ageofempires.fandom.com/wiki/Ship_(Age_of_Empires_II)'
soup_fish = get_data_from_wiki(url)

# Buildings
url = 'https://ageofempires.fandom.com/wiki/Buildings_(Age_of_Empires_II)'
soup_building = get_data_from_wiki(url)

# Tech
url = 'https://ageofempires.fandom.com/wiki/Technology_(Age_of_Empires_II)'
soup_tech = get_data_from_wiki(url)

# Create a dictionary to store the soup objects
soup_dict = {
    'infantry': soup_infantry,
    'archer': soup_archer,
    'cavalry': soup_cavalry,
    'fish': soup_fish,
    'building': soup_building,
    'tech': soup_tech
}

# save the dictionary to a pickle file
util.save_obj(soup_dict, '../data/soup_dict')

In [5]:
# This cell gets the soup dictionary and creates the individual soup objects in the environment 
soup_dict = util.load_obj('../data/soup_dict')
# Create the soup objects
for k,v in soup_dict.items():
    globals()[f'soup_{k}'] = v

The next cell gets all the units tables from https://ageofempires.fandom.com/wiki/Category:Age_of_Empires_II and concatenate them all together. 

I had to add the villager unit manually.


In [46]:
# Infantry
df_infantry = get_table(soup_infantry)

# Archers
df_archer = get_table(soup_archer)

# Cavalry
df_cav = get_table(soup_cavalry)

# Navy
df_navy = get_table(soup_fish)
# building column is missing from the navy data frame
df_navy['building'] = 'Dock'

# combine all data frames
df_units = pd.concat([df_infantry, df_archer, df_cav, df_navy]) 

# replace the empty cells for resources with 0s
df_units[['food', 'wood', 'gold']] = df_units[['food', 'wood', 'gold']].fillna(0)
# The stone column is missing because no unit cost stone to be trained
df_units['stone'] = 0
# place 'stone' column after 'gold' column
util.move_column(df_units, cols_to_move= ['stone'], ref_col = 'gold', relative_position = 'after')

# add row with Unit Villager to the data frame
df_vil = pd.DataFrame({'building': ['Town Center'], 'age': [''], 'food': [50], 'wood': [0], 'gold': [0], 'hp': [25], 'attack': [3], 'range': [0], 'rate of fire': [2], 'melee armor': [0], 'pierce armor': [0], 'speed': [0.8], 'line of sight': [4], 'training time': [25], 'civilization availability': ['All']}, index = ['villager'])
df_units = pd.concat([
    df_units,
    df_vil
])

print(df_units.tail(2))

# convert important columns to numeric
def transform_column(value):
    return str(value).split('/')[0] if '/' in str(value) else str(value)
columns_to_transform = ['training time', 'food', 'gold', 'wood']
df_units[columns_to_transform] = df_units[columns_to_transform].applymap(transform_column)
df_units[columns_to_transform] = df_units[columns_to_transform].apply(pd.to_numeric, errors='coerce')

df_units.to_csv('../data/unit_statistics.csv', index=True)




               building age   hp attack melee armor pierce armor range   
thirisadai         Dock      250      9           2           10     6  \
villager    Town Center       25      3           0            0     0   

           rate of fire line of sight speed food wood gold  stone   
thirisadai         3.45             8   1.3    0  180   60    0.0  \
villager              2             4   0.8   50    0    0    NaN   

           training time civilization availability frame delay accuracy  
thirisadai            40                Dravidians         NaN      NaN  
villager              25                       All         NaN      NaN  


In [48]:
# Buildings
df_building = get_table(soup_building)

df_building.to_csv('../data/building_statistics.csv', index=True)

print(df_building.tail(3))

             age     hp garrison capacity melee armor pierce armor   
building                                                             
krepost           2,600                20           8           11  \
feitoria          2,700                 0           3           10   
caravanserai      2,700                 0           3           10   

             line of sight wood stone gold construction time size   
building                                                            
krepost                 10    0   350    0               150  3×3  \
feitoria                 6    0   250  250               120  5×5   
caravanserai             6  175     0   50                60  4×4   

             civilization availability  
building                                
krepost                     Bulgarians  
feitoria                    Portuguese  
caravanserai               Hindustanis  


In [62]:
df_building

,age,hp,garrison capacity,melee armor,pierce armor,line of sight,wood,stone,gold,construction time,size,civilization availability
building,,,,,,,,,,,,
town center,,"2,400",15,3/4/5/6,5/6/7/8,8,275,100,0,150,4×4,All
house,,550/750/900/900,0,-2/-1/1/3,7/8/9/10,2,25,0,0,25,2×2,All exceptHuns
mill,,"600/800/1,000/1,000",0,0/1/2/3,7/8/9/10,6,100,0,0,35,2×2,All
farm,,480,0,0,0,1,60,0,0,15,3×3,All
lumber camp,,"600/800/1,000/1,000",0,0/1/2/3,7/8/9/10,6,100,0,0,35,2×2,All
mining camp,,"600/800/1,000/1,000",0,0/1/2/3,7/8/9/10,6,100,0,0,35,2×2,All
dock,,"1,800",10,0/1/2/3,7/8/9/10,6,150,0,0,35,3×3,All
fish trap,,50,0,0,0,1,100,0,0,40,1×1,All
market,,"1,800/2,100/2,100",0,1/2/3,8/9/10,6,175,0,0,60,4×4,All


For the Technology table, the logic to get the info doesn't work like the units and building tables. So we need to write something a bit different. Since I am only using it once, I won't bother writing it down as a function.

In [63]:
# Find the table based on its class or other attributes
# Get all tables in the webpage
all_tables = soup_tech.find_all('table', {'class': 'wikitable'})

# store the unstructured data
rows_list = []
for table in all_tables:
    rows = []
    for row in table.find_all('tr'):
        data = [cell.text.strip() for cell in row.find_all('td')]
        if data:
            rows.append(data)
    rows_list.append(rows)

# storage for the data frames representing each table with technology details
df_list = []  
for rows in rows_list:
    header = rows[0]
    len_header = len(header)
    for idx in range(1, len(rows)):
        row = rows[idx]
        len_row = len(row)
        # In that case it must be the case that the building is shared across rows but the effect is different across rows
        if len_row == len_header - 1:
            building = rows[idx-1][2]
            rows[idx].insert(2, building)
        # In that case it must be the case that the building and the effect are shared across rows
        if len_row == len_header - 2:
            building = rows[idx-1][2]
            effect = rows[idx-1][4]
            rows[idx].insert(2, building)
            rows[idx].insert(4, effect)

    df_t = pd.DataFrame(rows[1:], columns=header) #skip the first row because it is the header
    df_list.append(df_t)
    
# Concatenate all the data frames
# not all tables have relevant techonologies, restricting to the first 7 tables
df_tech = pd.concat(df_list[0:7])

# final header I want for the table
header_row = ['technology', 'age', 'building', 'cost_join', 'effect', 'research time']
df_tech.columns = header_row  

# the cost is saved as a string, I want to split it into food, wood, gold
# Define a function to extract resource cost
def extract_cost(resource, cost_str):
    pattern = f'(\d+){resource}'
    match = re.search(pattern, cost_str)
    return int(match.group(1)) if match else 0

# Create separate columns for each resource
df_tech['food']  = df_tech['cost_join'].apply(lambda x: extract_cost('F', x))
df_tech['wood']  = df_tech['cost_join'].apply(lambda x: extract_cost('W', x))
df_tech['gold']  = df_tech['cost_join'].apply(lambda x: extract_cost('G', x))
df_tech['stone'] = df_tech['cost_join'].apply(lambda x: extract_cost('S', x))

# Make the first column the index. The index will be the name of the unit/building
df_tech.set_index(df_tech.columns[0], inplace=True)
# Make index lower case
df_tech.index = df_tech.index.str.lower()

# convert research time to numeric
df_tech['research time'] = pd.to_numeric(df_tech['research time'], errors='coerce')

df_tech.head(10)

df_tech.to_csv('../data/technology_statistics.csv', index=False)

In [64]:
df_tech.head(10)


,age,building,cost_join,effect,research time,food,wood,gold,stone
technology,,,,,,,,,
town watch,,Town Center,75F,Buildings +4 Line of Sight,25,75,0,0,0
town patrol,,Town Center,300F 100G,Buildings +4 Line of Sight,40,300,0,100,0
treadmill crane,,University,300F 200W,Villagers build 20% faster,50,300,200,0,0
masonry,,University,150F 175W,"Buildings +10% hit points, +1/+1 armor, and +3...",50,150,175,0,0
architecture,,University,300F 200W,"Buildings +10% hit points, +1/+1 armor, and +3...",70,300,200,0,0
hoardings,,Castle,400F 400W,Castle HP +21%,75,400,400,0,0
conscription,,Castle,150F 150G,Work rate of military buildings (except Siege ...,60,150,0,150,0
wheelbarrow,,Town Center,175F 50W,"Villager +10% speed, +25% carrying capacity",75,175,50,0,0
hand cart,,Town Center,300F 200W,"Villager +10% speed +50%, carrying capacity",55,300,200,0,0


In [66]:
for col_name in df_tech.columns:
    col = df_tech[col_name]
    print(col.dtype)

object
object
object
object
object
int64
int64
int64
int64
